In [2]:
!pip install opencv-python-headless
!pip install opencv-contrib-python


In [6]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# Initialize OpenCV's face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces and extract the region of interest (ROI)
def extract_face(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image {image_path}")
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        print(f"No face detected in image {image_path}")
        return None  # No faces found

    x, y, w, h = faces[0]  # Take the first detected face
    face = img[y:y+h, x:x+w]
    return face

# Prepare data: Extract faces and labels from folder
def prepare_data(data_directory):
    faces = []
    labels = []
    label_names = os.listdir(data_directory)

    for label_name in label_names:
        label_dir = os.path.join(data_directory, label_name)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                face = extract_face(img_path)
                if face is not None:
                    faces.append(cv2.cvtColor(face, cv2.COLOR_BGR2GRAY))  # Convert face to grayscale
                    labels.append(label_name)
                else:
                    print(f"No face found in {img_path}")

    print(f"Total faces detected: {len(faces)}")  # Debugging line
    return np.array(faces), np.array(labels)

# Load data (Assuming the folder contains images of 'Person 1' and 'Person 2')
data_directory = '/content/DATA'  # Path to your dataset
faces, labels = prepare_data(data_directory)

if faces.shape[0] == 0:
    raise ValueError("No faces detected in the dataset. Please check the dataset and try again.")

# Encode labels to integers (Binary classification: Person1, Person2)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Normalize the pixel values to [0, 1]
faces = faces / 255.0
print(f"Shape of first face: {faces[0].shape}")  # Debugging line

# Reshape for CNN input
faces = faces.reshape(faces.shape[0], faces.shape[1], faces.shape[2], 1)  # Reshape for CNN input

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(faces, encoded_labels, test_size=0.2, random_state=42)

# Build the CNN model for binary classification
model = Sequential()
model.add(Flatten(input_shape=(faces.shape[1], faces.shape[2], 1)))  # Flatten the 2D faces to 1D
model.add(Dense(64, activation='relu'))  # First dense layer
model.add(Dense(1, activation='sigmoid'))  # Binary output (Person1 or Person2)

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Test the model with a new image
test_image_path = '/content/P2.jpg'  # Path to a test image
test_face = extract_face(test_image_path)

if test_face is not None:
    test_face = cv2.cvtColor(test_face, cv2.COLOR_BGR2GRAY)
    test_face = test_face / 255.0
    test_face = test_face.reshape(1, test_face.shape[0], test_face.shape[1], 1)  # Reshape for CNN
    prediction = model.predict(test_face)

    predicted_label = 'Person 1' if prediction[0] < 0.5 else 'Person 2'
    print(f'Predicted label: {predicted_label}')
else:
    print("No face detected in the test image.")


Total faces detected: 0


ValueError: No faces detected in the dataset. Please check the dataset and try again.